# Single responsibility principle

## Bad example (violates SRP)

In [ ]:
class Report:
    def __init__(self, text):
        self.text = text

    def print_report(self): #handles printing 
        print(self.text)
    
    def save_to_file (self, filename): # handles saving
        with open(filename, "w") as f:
            f.write(self.text)
    """
    What is wrong?
    Report is 
    1- storing the data
    2- printing the report
    3- saving the report to a file

    3 responsibilities in 1 class
    if printing changes, the class must change
    if saving method chages (e.g., save to DB), the class must change
    violates SRP
    """

## Good Example (applies SRP)

In [ ]:
class Report:
    def __init__ (self, text):
        self.text = text

class ReportPrinter:
    def print(self, report):
        print(report.text)

class ReportSaver:
    def save(self, report, filename):
        with open(filename, "w") as f:
            f.write (report.text)

"""
Now:
Report: only holds data
ReportPrinter: only prints
ReportSaver: only saves
each class has one reason to change
"""

In [ ]:
class Order:
    def __init__(self,items):
        self.items = items

class OrderCalculator:
    def calculate_total(self, order):
        #self.item = Order.item
        return sum(order.items)
    
class PrintReceipt:
    def print_receipt(self,order,calculator ):
        i = 1
        for item in order.items:
            print(f"Item {i}: {item}")
            i = i + 1
        print(f"Total: {calculator.calculate_total(order)}")

class SaveOrder:
    def save_order(self,filename, order, calculator):
        with open(filename, "w") as f:
            f.write("Order Details:\n")
            for item in order.items:
                f.write(f"{item}\n")
            f.write(f"Total: {calculator.calculate_total(order)}")

In [3]:
order1 =  Order((12,5,21,5))
calculator = OrderCalculator()
calc_order1 = calculator.calculate_total(order1)
PrintReceipt1 = PrintReceipt()
PrintReceipt1.print_receipt(order1,calculator= OrderCalculator())

Item 1: 12
Item 2: 5
Item 3: 21
Item 4: 5
Total: 43


# Open closed principle

## Bad Example (Violates OCP)

In [ ]:
class DiscountCalculator:
    def calculator (self, price, customer_type):
        if customer_type == "regular":
            return price
        elif customer_type == "vip":
            return price * 0.8 
        elif customer_type == "student":
            return price * 0.9
# every time you add a new customer type, you must modify this class --> BAD OCP

## Good example (Respects OCP)

### OCP using Abstraction with inheritance/polymorphism

In [2]:
from abc import ABC, abstractmethod
class DiscountStrategy(ABC):
    @abstractmethod
    def apply(self, price):
        pass

class RegularDiscount (DiscountStrategy):
    def apply(self, price):
        return price
    
class VIPDsicount(DiscountStrategy):
    def apply(self, price):
        return price * 0.8
    
class StudentDiscount (DiscountStrategy):
    def apply(self, price):
        return price * 0.9
    
class DiscountCalculator:
    def calculator(self,price,discount_stratgy : DiscountStrategy):
        return discount_stratgy.apply(price)
    
calculator = DiscountCalculator()
print (calculator.calculator(200, VIPDsicount()))


160.0


In [ ]:
class PaymentProcessor(ABC):
    @abstractmethod
    def processor_payment(self, amount):
        pass

class CreditCardPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing credit card payment and the amount is {amount}")

class PayPalPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing PayPal payment and the amount is {amount}")

class BankTransferPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing bank transfer and the amount is {amount}")

class PaymentService:
    def __init__(self,Payment_method :PaymentProcessor ):
        self.payment_method = Payment_method
    def pay(self,amount):
        self.payment_method.processor_payment(amount)
"""
1. Parameter name

payment_method → this is just the variable name.

2. Colon :

Means: “the expected type is…”

3. PaymentProcessor

This is the type hint:
We expect payment_method to be an instance of a class that inherits from(PaymentProcessor)
"""


In [4]:
payment_method = PayPalPayment()
service = PaymentService(payment_method)
service.pay(500)

Processing PayPal payment and the amount is 500


### OCP using composition 

In [ ]:
class Order:
    def __init__(self, items, calculator):
        self.items = items
        self.calculator = calculator
    def total(self):
        return self.calculator.calculate_total(self) 
"""
here you can create new calculator classes without modifying Order and you can extend behavior by passing new strategies
this uses the strategy pattern, which is another OCP technique
"""

# Liskov subistitution principle

## Bad Example (Violates LSP)

In [ ]:
class Bird:
    def fly(self):
        return "Flying!"
    
class Penguin (Bird):
    def fly(self):
        raise Exception("Penguins can't fly!")
    
def make_bird_fly(bird: Bird):
    print(bird.fly())

# make_bird_fly(Penguin()) #this line will break the code



Exception: Penguins can't fly!

In [ ]:
class Rectangle:
    def __init__(self, w, h):
        self.width = w
        self.height = h

    def set_width (self, w):
        self.width = w

    def set_height (self, h):
        self.height = h

class Square(Rectangle):
    def set_width(self, w):
        self.width = w
        self.height = w # force height = width 

    def set_height (self, h):
        self.height = h
        self.width = h # force width = hight 

In [ ]:
# If the subclass requires more conditions, LSP is violated
class Payment:
    def pay(self, amount):
        print(f"Paying {amount}")

class SecurePayment (Payment):
    def pay(self, amount, password): # Requires more input which violates LSP
        if password != "1234":
            raise Exception("Invalid password")
        print(f"Securely paying {amount}")

In [ ]:
# If the subclass does less conditions, LSP is violated
class Animal:
    def make_sound (self):
        print("Some generic sound")

class Snake (Animal):
    def make_sound(self):
        pass # does nothing     

## Good Example (Respects LSP)

In [3]:
class Bird:
    pass

class FlyingBird(Bird):
    def fly(self):
        return "Flying!"
    
class Penguin (Bird):
    def swim (self):
        return "Swimming!"

## Example

In [4]:
from abc import ABC, abstractmethod

class File(ABC):
    @abstractmethod
    def open(self):
        pass


class Writable(ABC):
    @abstractmethod
    def write(self, data):
        pass


class ReadOnlyFile(File):
    def open(self):
        print("Opening read-only file...")


class ReadWriteFile(File, Writable):
    def open(self):
        print("Opening read-write file...")

    def write(self, data):
        print(f"Writing: {data}")
